# Python Testing Demo

## Why testing is valuable in Python

## Different methods and packages for testing in Python

## Some tips & tricks

## Some gotchas

## Why testing is valuable in Python

- Proof that you delivered what was expected from you
- Prevent unexpected behavior
- Document both the functionality as well as the behavior of the code
- Make sure that what worked still works after code changes


In [ ]:
# /python_testing_demo/some_module_file.py

def function_that_returns_text(text: str):
    if isinstance(text, str):
        return text
    else:
        raise ValueError(f"Input did not have a '{str}' type, but '{type(text)}'!")

print(function_that_returns_text("You should get this text back!"))

This code does exactly what it says it does, but you *proof* that it will by using tests to describe its behavior.

In [ ]:
# Set up PyTest
!pip install ipytest
import ipytest
ipytest.autoconfig()

In [ ]:
# /tests/some_tests_file.py
def test_str():
    assert function_that_returns_text("You should see this text!") == "You should see this text!"

def test_int_as_str():
    assert function_that_returns_text("15") == "15"

def test_int():
    assert function_that_returns_text(15) == "15"

In [ ]:
ipytest.run('-vv')

### HTTP Responses

Sometimes you'll need to verify that certain HTTP requests contain specific data, i.e. to verify certain third-party behavior. You can catch and expose HTTP request data with the following code.

In [ ]:

# /tests/conftest.py
import logging
import pytest

@pytest.fixture
def debug_http(scope="module", autouse=False):
    """
    Catches all out-going HTTP calls and prints them in the console.
    Use `autouse=True` to automatically use it in every test which imports this conftest.py file.
    """
    import http.client as http_client

    http_client.HTTPConnection.debuglevel = 1
    logging.basicConfig()
    logging.getLogger("debug_http").setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

In [ ]:
# /tests/some_tests_file.py
%%ipytest -s -vvv -k test_http_logs_are_not_catched
import requests

def test_http_logs_are_not_catched(
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" not in out

ipytest.run('-vv')

In [ ]:
# /tests/some_tests_file.py
%%ipytest -s -vvv -k test_http_logs_are_catched

def test_http_logs_are_catched(
    debug_http,
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" not in out

ipytest.run('-vv')

In [ ]:
# /tests/conftest.py
import pytest
def pytest_configure():
    """
    Set up pytest global variables.
    """
    pytest.GLOBAL_TEST_VARIABLE = "test global variable"

In [ ]:
%%ipytest -k test_should_have_access_to_global_variable

# /tests/some_tests_file.py
import pytest


# TODO: Test fails, might need to move everything to actual tests/ folder and then execute/include from here?
def test_should_have_access_to_global_variable():
    assert pytest.GLOBAL_TEST_VARIABLE == "test global variable"

ipytest.run('-vv')